In [4]:
import numpy as np
import face_recognition
import os
import cv2

In [8]:
known_face_encodings = []
known_face_names = []


known_faces_dir = 'known_faces'
for filename in os.listdir(known_faces_dir):
    if filename.endswith(('.jpg', '.png', '.jpeg')):
        image_path = os.path.join(known_faces_dir, filename)
        print(image_path)
        image = face_recognition.load_image_file(image_path)
        encoding = face_recognition.face_encodings(image)[0]

        known_face_encodings.append(encoding)
        known_face_names.append(os.path.splitext(filename)[0])

print(known_face_names)

known_faces\umar.png
['umar']


In [9]:
video_capture = cv2.VideoCapture(0)

while True:
    ret, frame = video_capture.read()
    if not ret:
        break
    
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations) if face_locations else []

    for face_encoding, (top, right, bottom, left) in zip(face_encodings, face_locations):
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Not a User"

        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        if face_distances.size > 0:
            best_match_index = face_distances.argmin()
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
            else:
                name = "Not a User"

        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        if name == "Not a User":
            print("User not recognized. Do you want to add your name? (y/n)")
            if input().lower() == 'y':
                user_name = input("Enter your name: ")

                user_image_path = os.path.join(known_faces_dir, f"{user_name}.jpg")
                cv2.imwrite(user_image_path, rgb_frame[top:bottom, left:right])

                known_face_encodings.append(face_encoding)
                known_face_names.append(user_name)
 
                print(f"New user '{user_name}' added successfully.")

    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()
